In [2]:
import pickle
import pandas as pd
top = pd.read_csv('top_final.csv')

#basline_mape <10% и низкая дисперсия в цене
simple_category = ['Зуб 1085.52.05-1', 'Вал 3519.05.02.083','Вал 3572.05.10.200','Засов 3532.01.02.007',
                  'Засов 3532.01.02.007', 'Вал 3572.05.10.150', 'Сателлит 3536.11.01.028', 'Колесо 3519.05.02.061',
                  'Вал 3519.05.02.007', 'Ось 3536.03.00.001', 'Коромысло 3537.25.00.500 ']

#catboost_model < 10%
model_category = ['Колесо 3519.05.02.006','Вал 3536.11.01.002','Ось 3519.21.00.025','Коромысло 3519.21.00.023',
                 'ВАЛ-ШЕСТЕРНЯ 3536.11.01.002 ','Вал 3532.10.01.011', 'Колесо 3519.05.02.003',
                 'Ось 3519.05.02.081']
#высокая дисперсия и ошибка более 10% везде
unreal_category = ['Вант стрелы 00.1606.49.1 ','Ось 3519.05.02.082', ]

Сделаем прототип пользовательского меню. Мы успели предрасчитать стоимость закзаов только на сегодняшнюю дату

In [13]:

def explane_cost(name):


    print(name)
    if name not in list(top['товары']):
        print('Ошибка в названии товара')
        return None
    if name in unreal_category:
        print('К сожалению по этой категории мы не можем сделать предсказание')
        return None
    if name in simple_category:
        print('В данной категории низкий разброс цен и можно предсказывать по предыдущей цене')
        print('Стоимость предыдущей закупки данного товара:', top[top['товары']==name]['last_price'])
        print('Примерная стоимость в диапазоне процентов',top[top['товары']==name]['baseline_std_mape'])
        return None
    if name in model_category:
        print('Мы используем модель для предскзания данной категории')
        model = pickle.load(open('models/'+name+'.pickle', 'rb'))
        print('Стоимость предыдущей закупки данного товара:', top[top['товары']==name]['last_price'])
        print('Рекомендуемая стоимость закупки сегодня',top[top['товары']==name]['today_prediction'] )
        print('Топ 5 Признаков используемые моделью', model.get_feature_importance(prettified=True)[:5]['Feature Id'] )
        return None
    return False
    

In [17]:
#примеры использования
explane_cost('Ось 3536.03.00.001')

Ось 3536.03.00.001
В данной категории низкий разброс цен и можно предсказывать по предыдущей цене
Стоимость предыдущей закупки данного товара: 18    251592.356688
Name: last_price, dtype: float64
Примерная стоимость в диапазоне процентов 18    9.804882
Name: baseline_std_mape, dtype: float64


In [18]:
# 'Введите название товара, например: 'ВАЛ-ШЕСТЕРНЯ 3536.11.01.002 '
explane_cost('Колесо 3519.05.02.003')

Колесо 3519.05.02.003
Мы используем модель для предскзания данной категории
Стоимость предыдущей закупки данного товара: 13    279490.44586
Name: last_price, dtype: float64
Рекомендуемая стоимость закупки сегодня 13    283549.747712
Name: today_prediction, dtype: float64
Топ 5 Признаков используемые моделью 0       JTLINFRA.BO_max_last_30
1    GSCLCEMENT.BO_max_last_100
2             SCHN_mean_last_30
3              CRH_max_last_100
4               LAC_min_last_30
Name: Feature Id, dtype: object
